In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/exam-cheating-dataset/ExamCheatingDataset/test/images/good438_jpg.rf.a42568de33d644c77a26fa2a361905a3.jpg
/kaggle/input/exam-cheating-dataset/ExamCheatingDataset/test/images/cheat889_jpg.rf.cf9c4450f9ed72fead38ab8713b94b88.jpg
/kaggle/input/exam-cheating-dataset/ExamCheatingDataset/test/images/cheat08_jpg.rf.cc897207ceab68aab4485dc9635c38e5.jpg
/kaggle/input/exam-cheating-dataset/ExamCheatingDataset/test/images/good314_jpg.rf.b6c1fc75a55df97ba05242228a953413.jpg
/kaggle/input/exam-cheating-dataset/ExamCheatingDataset/test/images/2023-03-14-11-10-11_mp4-219_jpg.rf.8ce83b2522c829c08e8136017888e9a2.jpg
/kaggle/input/exam-cheating-dataset/ExamCheatingDataset/test/images/2023-03-14-11-10-11_mp4-25_jpg.rf.c75ffe2c320983d51b0270843da05f8d.jpg
/kaggle/input/exam-cheating-dataset/ExamCheatingDataset/test/images/youtube-90_jpg.rf.7b2a904232acef65be01baa9c72c2f73.jpg
/kaggle/input/exam-cheating-dataset/ExamCheatingDataset/test/images/youtube-338_jpg.rf.7b219d5dd50a43117746b047f2fff0

In [2]:
import cv2
import math
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dense, Flatten,Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense, LSTM, TimeDistributed, GlobalAveragePooling2D, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


In [3]:
data_path = '/kaggle/input/exam-cheating-dataset/ExamCheatingDataset/train/'
(os.listdir('/kaggle/input/exam-cheating-dataset/ExamCheatingDataset/train/'))
y=[]
for i in range(0,5):
    y.append(os.listdir('/kaggle/input/exam-cheating-dataset/ExamCheatingDataset/train/')[i])
print(y)

['giving object', 'giving code', 'normal act', 'cheating', 'looking friend']


In [14]:
dimension_1 = []
dimension_2 = []
base_path = '/kaggle/input/exam-cheating-dataset/ExamCheatingDataset/train/cheating'


for image_name in os.listdir(base_path):
    img_path = os.path.join(base_path, image_name)
    
   
    img = cv2.imread(img_path)
    
    if img is not None:
        # Get the dimensions of the image
        d1, d2, d3 = img.shape
        dimension_1.append(d1)
        dimension_2.append(d2)
    else:
        print(f"Error loading image: {img_path}")

d1 = np.mean(dimension_1)
d2 = np.mean(dimension_2)

img_shape = (math.ceil(d1),math.ceil(d1),d3)
print(d1,d2)

631.4166666666666 606.25


In [ ]:

Cheating_img_1 = cv2.imread(os.path.join(base_path,'63_jpeg_jpg.rf.91de2873a56f522f7ac22b2cfe2e859f.jpg'))
plt.imshow(Cheating_img_1)
Cheating_img_1.max()

255

In [ ]:
imageGen = ImageDataGenerator(rotation_range = 15,
                              width_shift_range = 0.1,
                              height_shift_range = 0.1,
                              shear_range = 0.08,
                              zoom_range = 0.1,
                              horizontal_flip = True,
                              fill_mode = 'nearest',
                              rescale = 1/255
)

In [7]:
base_path = '/kaggle/input/exam-cheating-dataset/ExamCheatingDataset/train/'
imageGen.flow_from_directory(base_path)
print(os.listdir(base_path))
class_indices = {'cheating': 0, 'giving code': 1, 'giving object': 2,'looking friend':3,'normal act':4}  


train_generator = imageGen.flow_from_directory(
    base_path,
    target_size=(math.ceil(d1),math.ceil(d1)),
    color_mode = 'rgb',
    batch_size=32,
    shuffle=True,
    classes=list(class_indices.keys()),  
    class_mode='categorical'
)

Found 1562 images belonging to 5 classes.
['giving object', 'giving code', 'normal act', 'cheating', 'looking friend']
Found 1562 images belonging to 5 classes.


In [8]:
data, labels = [], []
for i in range(len(train_generator)):
    batch = next(train_generator)
    data.extend(batch[0])
    labels.extend(batch[1])
    if len(data) >= train_generator.samples:
        break

data = np.array(data)
labels = np.array(labels)

X_train_and_val, X_test, y_train_and_val, y_test = train_test_split(data, labels, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_and_val, y_train_and_val, test_size=0.1, random_state=42)

print(f"Number of training samples: {len(X_train)}")
print(f"Number of validation samples: {len(X_val)}")
print(f"Number of test samples: {len(X_test)}")

Number of training samples: 1264
Number of validation samples: 141
Number of test samples: 157


In [9]:
train_generator = imageGen.flow(X_train, y_train, batch_size=32)
val_generator = ImageDataGenerator().flow(X_val, y_val, batch_size=32)
test_generator = ImageDataGenerator().flow(X_test, y_test, batch_size=32)


batch_images, batch_labels = next(val_generator)
print("Minimum pixel value:", batch_images.min())
print("Maximum pixel value:", batch_images.max())

Minimum pixel value: 0.0
Maximum pixel value: 1.0


In [38]:


model = Sequential([

    Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(134,134,3), padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.2),
    
    Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.3),
    
    Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.3),
    
    Conv2D(256, kernel_size=(3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.4),
    
    Flatten(),
    
    Reshape((-1,16384)), 
    
 
    LSTM(128, return_sequences=False),
    Dropout(0.5),
    

    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(5, activation='softmax')  
])

model.compile(
    optimizer=Adam(learning_rate=0.0001),  
    loss='categorical_crossentropy',
    metrics=['accuracy']  
)



In [39]:

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

model_checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5, 
    patience=2,
    min_lr=1e-6,
    verbose=1
)

history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // 32,
    validation_data=val_generator,
    validation_steps=len(X_val) // 32,
    epochs=50,
    callbacks=[early_stop, model_checkpoint]
)

model.load_weights('best_model.keras')

loss, accuracy = model.evaluate(test_generator, verbose=2)
print(f"Test Accuracy: {accuracy:.4f}")

print("Training Finished")

Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node sequential_22_1/reshape_20_1/Reshape defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/lo

In [ ]:
model.save('Train_model_1.keras')